# Spatial Correspondance Using Ray Casting of Ischemic and Territory Regions on the Surface of the Myocardium

Begin by specifying the path to the directory that contains the data required for ray-based analysis. This directory must include the following five files:

 1. MBF Model (VTU format): A volumetric model extracted from the MBF (Myocardial Blood Flow) map.

 2. Basal Slice (VTP format): A slice of the MBF model captured around the base of the myocardium.

 3. Apical Slice (VTP format): A slice of the MBF model representing the apical hemisphere.

 4. Ischemic Region (VTU format): A segmentation of the ischemic region associated with the vessel of interest, also extracted from the MBF map.

 5. MBF Territories Labels (dat format): A text file including the name tag and the number tag of the territories assigned to each branch.

Also, the extracted surface of the MBF map model should be saved into the parent directory of the vessel specific working directory.

After providing the input files, configure the TerritoryTags. These tags indicate myocardial regions downstream of a stenosis, based on the output file MBF_Territories.vtu.

Finally, ensure that the image scale is set appropriately—either in centimeters (cm) or millimeters (mm)—to match the spacing of the input image data. Accurate scaling is essential for proper geometric and spatial interpretation.

In [ ]:
InputFolder = "/Users/ana/Documents/AnahitaSeresti/05_PrePostCABG/RayCasting/SU04B"
slice_apex = f"{InputFolder}/Slice_Apex.vtp"
slice_base = f"{InputFolder}/Slice_Base.vtp"
#InputMyocardium = f"{InputFolder}/Myocardium.vtp"
InputMBF = f"{InputFolder}/MBF_Map.vtu"
#InputMyocardiumVTU = f"{InputFolder}/MyocardiumModel.vtu"
OutputFolder = f"{InputFolder}/RayCastingResults"


scale = 'cm'

In [ ]:
import os
import vtk
import numpy as np
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
from utilities import ReadVTUFile, ReadVTPFile, WriteVTPFile, WriteVTUFile, GetCentroid, ThresholdPointsByUpper, LargestConnectedRegion, PrintProgress, ThresholdByUpper, ExtractSurface, ReadVTKFile

os.system(f"mkdir {OutputFolder}")

NRaySection = 1500
NRaySphere = 40000
NSection = 100


if scale == 'cm':
    Radius = 3.5
    L_Section_Ray = 10
    L_Sphere_Ray = 5
    Ray_Thickness = 0.02
    Base_Cover = 3.5
    Apex_Cover = 0

elif scale == 'mm':
    Radius = 35
    L_Section_Ray = 100
    L_Sphere_Ray = 50
    Ray_Thickness = 0.2
    Base_Cover = 35
    Apex_Cover = 0

else:
    print("provide a proper scale")



### Defining necessary functions

In [ ]:

def Line(point1, point2, res):
    line = vtk.vtkLineSource()
    line.SetPoint1(point1)
    line.SetPoint2(point2)
    line.SetResolution(res)
    line.Update()
    
    return line.GetOutput()

def BoldLine(centerline):
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(centerline)
    tube_filter.SetRadius(Ray_Thickness)  
    tube_filter.SetNumberOfSides(50)  
    tube_filter.CappingOn()  
    tube_filter.Update()
    
    return tube_filter.GetOutput()

def SliceWPlane(Volume,Origin,Norm):
    plane=vtk.vtkPlane()
    plane.SetOrigin(Origin)
    plane.SetNormal(Norm)
    Slice=vtk.vtkCutter()
    Slice.GenerateTrianglesOff()
    Slice.SetCutFunction(plane)
    Slice.SetInputData(Volume)
    Slice.Update()
    
    return Slice.GetOutput()

def ClipSurfaceWPlane(surface, center, axis, IO = False):
    plane = vtk.vtkPlane()
    plane.SetOrigin(center)
    plane.SetNormal(axis)

    clipper = vtk.vtkClipPolyData()
    clipper.SetInputData(surface)
    clipper.SetClipFunction(plane)
    clipper.GenerateClippedOutputOff()
    if IO:
        clipper.InsideOutOn()
    else:
        clipper.InsideOutOff()
    #clipper.GetOutputInformation(1)
    clipper.Update()

    return clipper.GetOutput()

def ClipVolumeWPlane(volume, center, axis, IO = False):
    plane = vtk.vtkPlane()
    plane.SetOrigin(center)
    plane.SetNormal(axis)

    clipper = vtk.vtkClipDataSet()
    clipper.SetInputData(volume)
    clipper.SetClipFunction(plane)
    if IO:
        clipper.InsideOutOn()
    else:
        clipper.InsideOutOff()

    clipper.Update()

    return clipper.GetOutput()

def RotateVector(ray, normal, angle):
    angle = np.radians(angle)
    normal /= np.linalg.norm(normal)
    ray_rot = (ray*np.cos(angle) + np.cross(normal, ray) * np.sin(angle) + normal * np.dot(normal, ray) * (1 - np.cos(angle)))
    
    return ray_rot

def ProbeFilter(InputData, SourceData):
    probe = vtk.vtkProbeFilter()
    probe.AddInputData(InputData)
    probe.SetSourceData(SourceData)
    probe.Update()
    
    return probe.GetOutput()

def IncreaseResolution(surface, subdivisions = 3):
    subdivide = vtk.vtkLoopSubdivisionFilter()
    subdivide.SetInputData(surface)
    subdivide.SetNumberOfSubdivisions(subdivisions)
    subdivide.Update()

    return subdivide.GetOutput()

def Cylinder(CenterLine):
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(CenterLine)
    tube_filter.SetRadius(Radius)
    tube_filter.SetNumberOfSides(2*NRaySection)
    tube_filter.CappingOff()
    tube_filter.Update()
    
    return tube_filter.GetOutput()

def Hemisphere(apex_centeroid, CL_axis):
    low_res = NRaySphere // 10
    
    sphere = vtk.vtkSphereSource()
    sphere.SetCenter(apex_centeroid)
    sphere.SetRadius(Radius)
    sphere.SetPhiResolution(low_res)
    sphere.SetThetaResolution(low_res)
    sphere.Update()

    Hemisphere = ClipSurfaceWPlane(sphere.GetOutput(), apex_centeroid, CL_axis)
    

    return Hemisphere

def fibonacci_sphere(samples=100):
    """Generates evenly distributed unit vectors over a sphere"""
    points = []
    phi = np.pi * (3. - np.sqrt(5.))  # Golden angle in radians

    for i in range(samples):
        y = 1 - (i / float(samples - 1)) * 2  # y goes from 1 to -1
        radius = np.sqrt(1 - y * y)  # Radius at given y
        theta = phi * i  # Golden angle increment

        x = np.cos(theta) * radius
        z = np.sin(theta) * radius
        points.append([x, y, z])

    return np.array(points)


In [ ]:
SliceApex = ReadVTPFile(slice_apex)
SliceBase = ReadVTPFile(slice_base)

base_centeroid = GetCentroid(SliceBase)
apex_centeroid = GetCentroid(SliceApex)

AnnotationPoints = [base_centeroid, apex_centeroid]

centerline_axis = np.array([AnnotationPoints[1][0] - AnnotationPoints[0][0], 
                                AnnotationPoints[1][1] - AnnotationPoints[0][1], 
                                AnnotationPoints[1][2] - AnnotationPoints[0][2]])

CL_axis = centerline_axis/np.linalg.norm(centerline_axis)

point0 = np.array([AnnotationPoints[0][0] - CL_axis[0]*Base_Cover,
                    AnnotationPoints[0][1] - CL_axis[1]*Base_Cover, 
                    AnnotationPoints[0][2] - CL_axis[2]*Base_Cover])

point1 = np.array([AnnotationPoints[1][0] + CL_axis[0]*Apex_Cover,
                    AnnotationPoints[1][1] + CL_axis[1]*Apex_Cover, 
                    AnnotationPoints[1][2] + CL_axis[2]*Apex_Cover])


In [ ]:
print("--- Extracting the Centerline of the Myocardium")
cl_file = f"{OutputFolder}/CenterLine.vtp"
CenterLine = Line(point0, point1, NSection)
WriteVTPFile(cl_file, BoldLine(CenterLine))

InputMBFBase = f"{OutputFolder}/UpperMBF.vtu"
InputMBFApex = f"{OutputFolder}/ApexMBF.vtu"


MBF_ = ReadVTUFile(InputMBF)
MBF = ClipVolumeWPlane(MBF_, apex_centeroid, CL_axis, True)
Apex = ClipVolumeWPlane(MBF_, apex_centeroid, CL_axis, False)
WriteVTUFile(InputMBFBase, MBF)
WriteVTUFile(InputMBFApex, Apex)

#MyocardiumSurface = ReadVTPFile(InputMyocardium)
'''
Myocardium = ReadVTUFile(InputMyocardiumVTU)

Boundary_Array = numpy_to_vtk(np.array([1 for _ in range(Myocardium.GetNumberOfPoints())]))
Boundary_Array.SetName("MyocardiumSilhouette")
Myocardium.GetPointData().AddArray(Boundary_Array)

InputMBFBase = f"{OutputFolder}/UpperMyo.vtu"
InputMBFApex = f"{OutputFolder}/ApexMyo.vtu"

"""ConvertVTU = vtk.vtkAppendFilter()
ConvertVTU.SetInputData(Myocardium)
ConvertVTU.Update()
MyocardiumVTU = ConvertVTU.GetOutput()"""
UpperMyo = ClipVolumeWPlane(Myocardium, apex_centeroid, CL_axis, True)
ApexMyo = ClipVolumeWPlane(Myocardium, apex_centeroid, CL_axis, False)
WriteVTUFile(InputMBFBase, UpperMyo)
WriteVTUFile(InputMBFApex, ApexMyo)
'''

In [ ]:
for i in range(MBF_.GetPointData().GetNumberOfArrays()):
    arrayname_ = MBF_.GetPointData().GetArrayName(i)
    if 'scalars' in arrayname_.lower():
        ScalarArray = arrayname_
    elif 'MappedScalar' in arrayname_:
        ScalarArray = "MappedScalar"

ScalarArray

In [ ]:
def CastRaysVisualizations(CLPoints):
    
    #>>> Ray Casting across the Myocardium Cylinder 
    slices = vtk.vtkAppendPolyData()

    NRays_sample = 20
    ray_ = np.array([-CL_axis[2], 0, CL_axis[0]])
    res = 100
    angles = np.linspace(0, 360, NRays_sample, endpoint= False)
    Rays = vtk.vtkAppendPolyData()
    
    for k in range(0,len(CLPoints), int(len(CLPoints)/(NSection/5))):
        MBF_slice = SliceWPlane(MBF, CLPoints[k], CL_axis)
        slices.AddInputData(MBF_slice)

        origin = CLPoints[k]
        for i in range(NRays_sample):
            ray_new = RotateVector(ray_, CL_axis, angles[i])
            point2 = np.array([origin[0]+ L_Section_Ray*ray_new[0], origin[1] + L_Section_Ray*ray_new[1], origin[2] + L_Section_Ray*ray_new[2]])
            ray = Line(origin, point2, res)
            ray_projected = ProbeFilter(ray, MBF)
            
            mbf_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray(ScalarArray))
            mbf_values = mbf_profile[mbf_profile > 0]
            average_mbf = np.mean(mbf_values)
            mbf_profile = numpy_to_vtk(np.array([average_mbf for _ in range(ray_projected.GetNumberOfPoints())]))
            
            mbf_profile.SetName("MBFProfile")
            

            territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
            territory = territory_profile[territory_profile > 0]
            
            if territory.size > 0:
                territory_tag = np.bincount(territory).argmax()
                territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
            else:
                territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
            
            territory_profile.SetName("TerritoryProfile")

            #ray_projected = ProbeFilter(ray_projected, UpperMyo)
            ray_projected.GetPointData().AddArray(mbf_profile)
            ray_projected.GetPointData().AddArray(territory_profile)
            '''
            Silhouette = vtk_to_numpy(ray_projected.GetPointData().GetArray("MyocardiumSilhouette"))

            if Silhouette is None or len(Silhouette) != ray_projected.GetNumberOfPoints():
                wall_thinkness = 0
            else:
                if np.any(Silhouette == 1):
                    indices_of_one = np.where(Silhouette == 1)[0]
                    wall_thinkness = L_Section_Ray*(indices_of_one[-1] - indices_of_one[0] + 1)/res
                else:
                    wall_thinkness = 0

            WallThicknessArray = numpy_to_vtk(np.full(ray_projected.GetNumberOfPoints(), wall_thinkness, dtype=float))
            WallThicknessArray.SetName("WallThickness")
            ray_projected.GetPointData().AddArray(WallThicknessArray)
            '''
            Rays.AddInputData(BoldLine(ray_projected))
    
        Rays.Update()


    #>>> Ray Casting across the Apex Hemisphere
    NRays_sample = 100
    directions = fibonacci_sphere(NRays_sample)
    origin = apex_centeroid
    Rays_ = vtk.vtkAppendPolyData()

    for i in range(NRays_sample):
        ray_new = directions[i]
        point2 = np.array([origin[0]+ L_Sphere_Ray*ray_new[0], origin[1] + L_Sphere_Ray*ray_new[1], origin[2] + L_Sphere_Ray*ray_new[2]])
        ray = Line(origin, point2, res)
        ray_projected = ProbeFilter(ray, Apex)
            
        mbf_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray(ScalarArray))
        mbf_values = mbf_profile[mbf_profile > 0]
        average_mbf = np.mean(mbf_values)
        mbf_profile = numpy_to_vtk(np.array([average_mbf for _ in range(ray_projected.GetNumberOfPoints())]))
        
        mbf_profile.SetName("MBFProfile")
        

        territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
        territory = territory_profile[territory_profile > 0]
        
        if territory.size > 0:
            territory_tag = np.bincount(territory).argmax()
            territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
        
        territory_profile.SetName("TerritoryProfile")

        #ray_projected = ProbeFilter(ray_projected, ApexMyo)
        ray_projected.GetPointData().AddArray(mbf_profile)
        ray_projected.GetPointData().AddArray(territory_profile)
        '''
        Silhouette = vtk_to_numpy(ray_projected.GetPointData().GetArray("MyocardiumSilhouette"))
        
        if Silhouette is None or len(Silhouette) != ray_projected.GetNumberOfPoints():
            wall_thinkness = 0
        else:
            if np.any(Silhouette == 1):
                indices_of_one = np.where(Silhouette == 1)[0]
                wall_thinkness = L_Section_Ray*(indices_of_one[-1] - indices_of_one[0] + 1)/res
            else:
                wall_thinkness = 0

        WallThicknessArray = numpy_to_vtk(np.full(ray_projected.GetNumberOfPoints(), wall_thinkness, dtype=float))
        WallThicknessArray.SetName("WallThickness")
        ray_projected.GetPointData().AddArray(WallThicknessArray)
        '''
        Rays_.AddInputData(BoldLine(ray_projected))


    Rays_.Update()
    Rays_ = ClipSurfaceWPlane(Rays_.GetOutput(), point1, CL_axis)

    Rays.AddInputData(Rays_)
    Rays.Update()

    ray_path = f"{OutputFolder}/Rays.vtp"
    WriteVTPFile(ray_path, Rays.GetOutput())

    slices.Update()
    slice_path = f"{OutputFolder}/MyoSlices.vtp"
    WriteVTPFile(slice_path, slices.GetOutput())


In [ ]:
def RayCastingAlongMyocardium(CLPoints):

    NRays = NRaySection
    ray_ = np.array([-CL_axis[2], 0, CL_axis[0]])
    res = 50
    angles = np.linspace(0, 360, NRays, endpoint= False)
    Rays = vtk.vtkAppendPolyData()
    
    print("------ Ray Casting across Sections along Myocardium:")
    progress_old = -1
    for k in range(len(CLPoints)):
        progress_old = PrintProgress(k,len(CLPoints),progress_old)

        origin = CLPoints[k]
        slice_rays = vtk.vtkAppendPolyData()
        
        for i in range(NRays):
            ray_new = RotateVector(ray_, CL_axis, angles[i])
            point2 = np.array([origin[0]+ L_Section_Ray*ray_new[0], origin[1] + L_Section_Ray*ray_new[1], origin[2] + L_Section_Ray*ray_new[2]])
            ray = Line(origin, point2, res)
            ray_projected = ProbeFilter(ray, MBF)
            
            mbf_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray(ScalarArray))
            #mbf_values = mbf_profile[mbf_profile > 0]
            average_mbf = np.mean(mbf_profile)
            mbf_profile = numpy_to_vtk(np.array([average_mbf for _ in range(ray_projected.GetNumberOfPoints())]))
            
            mbf_profile.SetName("MBFProfile")
            

            territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
            territory = territory_profile[territory_profile > 0]
            
            if territory.size > 0:
                territory_tag = np.bincount(territory).argmax()
                territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
            else:
                territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
            
            territory_profile.SetName("TerritoryProfile")

            #ray_projected = ProbeFilter(ray_projected, UpperMyo)
            ray_projected.GetPointData().AddArray(mbf_profile)
            ray_projected.GetPointData().AddArray(territory_profile)
            '''
            Silhouette = vtk_to_numpy(ray_projected.GetPointData().GetArray("MyocardiumSilhouette"))

            if Silhouette is None or len(Silhouette) != ray_projected.GetNumberOfPoints():
                wall_thinkness = 0
            else:
                if np.any(Silhouette == 1):
                    indices_of_one = np.where(Silhouette == 1)[0]
                    wall_thinkness = L_Section_Ray*(indices_of_one[-1] - indices_of_one[0] + 1)/res
                else:
                    wall_thinkness = 0

            WallThicknessArray = numpy_to_vtk(np.full(ray_projected.GetNumberOfPoints(), wall_thinkness, dtype=float))
            WallThicknessArray.SetName("WallThickness")
            ray_projected.GetPointData().AddArray(WallThicknessArray)
            '''
            slice_rays.AddInputData(ray_projected)
        slice_rays.Update()
    
        Rays.AddInputData(slice_rays.GetOutput())
    
    Rays.Update()

    return Rays.GetOutput()

In [ ]:
def RayCastingAcrossSphere():
    directions = fibonacci_sphere(NRaySphere)
    origin = apex_centeroid
    Rays_Hemisphere = vtk.vtkAppendPolyData()
    res = 50

    print("------ Ray Casting across the Apex Hemisphere:")
    progress_old = -1
    for i in range(NRaySphere):
        progress = PrintProgress(i, NRaySphere, progress_old)
        progress_old = progress

        ray_new = directions[i]
        point2 = np.array([origin[0]+ L_Sphere_Ray*ray_new[0], origin[1] + L_Sphere_Ray*ray_new[1], origin[2] + L_Sphere_Ray*ray_new[2]])
        ray = Line(origin, point2, res)
        ray_projected = ProbeFilter(ray, Apex)
            
        mbf_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray(ScalarArray))
        #mbf_values = mbf_profile[mbf_profile > 0]
        average_mbf = np.mean(mbf_profile)
        mbf_profile = numpy_to_vtk(np.array([average_mbf for _ in range(ray_projected.GetNumberOfPoints())]))
        
        mbf_profile.SetName("MBFProfile")
        

        territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
        territory = territory_profile[territory_profile > 0]
        
        if territory.size > 0:
            territory_tag = np.bincount(territory).argmax()
            territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
        
        territory_profile.SetName("TerritoryProfile")

        #ray_projected = ProbeFilter(ray_projected, ApexMyo)
        ray_projected.GetPointData().AddArray(mbf_profile)
        ray_projected.GetPointData().AddArray(territory_profile)
        '''
        Silhouette = vtk_to_numpy(ray_projected.GetPointData().GetArray("MyocardiumSilhouette"))
        
        if Silhouette is None or len(Silhouette) != ray_projected.GetNumberOfPoints():
            wall_thinkness = 0
        else:
            if np.any(Silhouette == 1):
                indices_of_one = np.where(Silhouette == 1)[0]
                wall_thinkness = L_Section_Ray*(indices_of_one[-1] - indices_of_one[0] + 1)/res
            else:
                wall_thinkness = 0

        WallThicknessArray = numpy_to_vtk(np.full(ray_projected.GetNumberOfPoints(), wall_thinkness, dtype=float))
        WallThicknessArray.SetName("WallThickness")
        ray_projected.GetPointData().AddArray(WallThicknessArray)
        '''
        Rays_Hemisphere.AddInputData(ray_projected)

    Rays_Hemisphere.Update()
    Rays_Hemisphere = ClipSurfaceWPlane(Rays_Hemisphere.GetOutput(), point1, CL_axis)

    return Rays_Hemisphere


### Implementing the Method

In [ ]:
print("--- Visualizing Ray Casting")
CLPoints = CenterLine.GetPoints()
CLPointsArray = np.array([CLPoints.GetPoint(i) for i in range(CLPoints.GetNumberOfPoints())])
CastRaysVisualizations(CLPointsArray)

## Verify Geometry and Coverage
Before proceeding, double-check the visuals:

 - Confirm that the centerline extends along the myocardium as expected.

 - Ensure that the rays generated from each centerline point adequately cover the myocardial wall in all directions.



In [ ]:
print("--- Ray Casting Across the Myocardium")
Rays_Myocardium = RayCastingAlongMyocardium(CLPointsArray)

In [ ]:
WriteVTPFile(f"{OutputFolder}/Rays_Myocardium.vtp", Rays_Myocardium)

In [ ]:
Rays_Hemisphere = RayCastingAcrossSphere()

In [ ]:
WriteVTPFile(f"{OutputFolder}/Rays_Hemisphere.vtp", Rays_Hemisphere)

In [ ]:
OutputSurface = vtk.vtkAppendPolyData()
OutputSurface.AddInputData(Rays_Myocardium)
OutputSurface.AddInputData(Rays_Hemisphere)
OutputSurface.Update()

OutputVolume_ = vtk.vtkAppendFilter()
OutputVolume_.AddInputData(OutputSurface.GetOutput())
OutputVolume_.Update()
OutputVolume = OutputVolume_.GetOutput()


In [ ]:
MyocardiumSurface = ReadVTPFile(f"{InputFolder}/Epicardium.vtp")


In [ ]:
MBFProfile_Array = vtk.vtkFloatArray()
MBFProfile_Array.SetName("MBFProfile")
MBFProfile_Array.SetNumberOfComponents(1)
MBFProfile_Array.SetNumberOfTuples(MyocardiumSurface.GetNumberOfPoints())

TerritoryProfile_Array = vtk.vtkFloatArray()
TerritoryProfile_Array.SetName("TerritoryProfile")
TerritoryProfile_Array.SetNumberOfComponents(1)
TerritoryProfile_Array.SetNumberOfTuples(MyocardiumSurface.GetNumberOfPoints())
'''
WallThickness_Array = vtk.vtkFloatArray()
WallThickness_Array.SetName("WallThickness")
WallThickness_Array.SetNumberOfComponents(1)
WallThickness_Array.SetNumberOfTuples(MyocardiumSurface.GetNumberOfPoints())
'''
Rays_Output = OutputVolume
Rays_MBF = Rays_Output.GetPointData().GetArray("MBFProfile")
Rays_territory = Rays_Output.GetPointData().GetArray("TerritoryProfile")
#Rays_thickness = Rays_Output.GetPointData().GetArray("WallThickness")

Locator = vtk.vtkPointLocator()
Locator.SetDataSet(Rays_Output)
Locator.BuildLocator()

tolerance = 1
dist2 = vtk.reference(0.0)


for i in range(MyocardiumSurface.GetNumberOfPoints()):
    point = MyocardiumSurface.GetPoint(i)
    closest_point_id = Locator.FindClosestPoint(point)
    
    if closest_point_id == -1:
        MBFProfile_Array.SetValue(i, 0)
        TerritoryProfile_Array.SetValue(i, -1)
        #WallThickness_Array.SetValue(i, 0)
    else:
        MBFProfile_Array.SetValue(i, Rays_MBF.GetValue(closest_point_id))
        TerritoryProfile_Array.SetValue(i, Rays_territory.GetValue(closest_point_id))
        #WallThickness_Array.SetValue(i, Rays_thickness.GetValue(closest_point_id))
    

MyocardiumSurface.GetPointData().AddArray(MBFProfile_Array)
MyocardiumSurface.GetPointData().AddArray(TerritoryProfile_Array)
#MyocardiumSurface.GetPointData().AddArray(WallThickness_Array)

WriteVTPFile(f"{InputFolder}/MyocardiumSurface.vtp", MyocardiumSurface)